In [ ]:
import os
import json
import re
from datetime import datetime

import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

import matplotlib.pyplot as plt

from pathlib import Path
from typing import List, Dict, Any, Iterable


In [1]:
# ------------------------------------------------------------------------------
# DTOF CNN – Post-processing and best-model extraction
#
# This script reads the JSON experiment log generated during the grid search and
# identifies the best-performing configurations for each evaluation metric. It
# reports:
#     • the run with the lowest validation MSE (best_val)
#     • the lowest MAE for μa and μs′
#     • the lowest RMSE for μa and μs′
#
# The purpose is to give a clean, at-a-glance summary of which preprocessing
# settings and hyperparameters work best for different reconstruction targets.
# Absorption and scattering errors behave differently across runs, so the “best”
# model depends on what matters most for the experiment. This script keeps the
# selection transparent and reproducible.
#
# It does not retrain anything — it simply parses the JSON log, scans the stored
# metrics, and prints the top configuration for each category.
# ------------------------------------------------------------------------------


In [ ]:
# JSON log path 
JSON_PATH = Path(
    "/Users/lydialichen/Library/CloudStorage/OneDrive-UniversityCollegeLondon/"
    "Year 3/Research Project in Biomedical Engineering/Code/JSON logs/dtof_runs_log.json"
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

Using device: cpu


In [14]:
JSON_PATH = Path(
    "/Users/lydialichen/Library/CloudStorage/OneDrive-UniversityCollegeLondon/"
    "Year 3/Research Project in Biomedical Engineering/Code/JSON logs/dtof_runs_log.json"
)


def load_run_logs(json_path: Path) -> List[Dict[str, Any]]:
    """
    Load DTOF run logs from a JSON file and normalise them into a list of dicts.
    """
    with json_path.open("r") as f:
        data = json.load(f)

    if isinstance(data, list):
        return data

    if isinstance(data, dict) and "runs" in data and isinstance(data["runs"], list):
        return data["runs"]

    if isinstance(data, dict):
        runs = [v for v in data.values() if isinstance(v, dict)]
        if runs:
            return runs

    raise ValueError("Unrecognised JSON structure for run logs.")


def print_global_best_metrics(runs: List[Dict[str, Any]]) -> None:
    """
    Find and print:
      - Best config by best_val (MSE)
      - Best config by MAE_μa, MAE_μs'
      - Best config by RMSE_μa, RMSE_μs'
    """

    best_mse_run = None
    best_mse_val = float("inf")

    best_mae_mua_run = None
    best_mae_mua_val = float("inf")

    best_mae_mus_run = None
    best_mae_mus_val = float("inf")

    best_rmse_mua_run = None
    best_rmse_mua_val = float("inf")

    best_rmse_mus_run = None
    best_rmse_mus_val = float("inf")

    for r in runs:
        # ----- MSE -----
        mse = r.get("best_val", None)
        if mse is not None and mse < best_mse_val:
            best_mse_val = mse
            best_mse_run = r

        mae = r.get("MAE", None)
        rmse = r.get("RMSE", None)

        # ----- MAE -----
        if mae is not None and len(mae) >= 2:
            mae_mua, mae_mus = float(mae[0]), float(mae[1])

            if mae_mua < best_mae_mua_val:
                best_mae_mua_val = mae_mua
                best_mae_mua_run = r

            if mae_mus < best_mae_mus_val:
                best_mae_mus_val = mae_mus
                best_mae_mus_run = r

        # ----- RMSE -----
        if rmse is not None and len(rmse) >= 2:
            rmse_mua, rmse_mus = float(rmse[0]), float(rmse[1])

            if rmse_mua < best_rmse_mua_val:
                best_rmse_mua_val = rmse_mua
                best_rmse_mua_run = r

            if rmse_mus < best_rmse_mus_val:
                best_rmse_mus_val = rmse_mus
                best_rmse_mus_run = r

    print("=" * 80)
    print("GLOBAL BEST CONFIGS BY METRIC")
    print("=" * 80)

    # Best MSE
    if best_mse_run is not None:
        print("\nBest by MSE (best_val):")
        print(f"  run_name: {best_mse_run['run_name']}")
        print(f"  channel_mode: {best_mse_run.get('channel_mode')}")
        print(f"  sg_window: {best_mse_run.get('sg_window')}, "
              f"sg_order: {best_mse_run.get('sg_order')}")
        print(f"  lr: {best_mse_run.get('lr')}")
        print(f"  best_val (MSE): {best_mse_val:.6f}")
    else:
        print("\nBest by MSE: (no runs found)")

    # Best MAE μa
    if best_mae_mua_run is not None:
        print("\nBest by MAE(μa):")
        print(f"  run_name: {best_mae_mua_run['run_name']}")
        print(f"  channel_mode: {best_mae_mua_run.get('channel_mode')}")
        print(f"  MAE(μa): {best_mae_mua_val:.6f}")
    else:
        print("\nBest by MAE(μa): (no runs with MAE)")

    # Best MAE μs'
    if best_mae_mus_run is not None:
        print("\nBest by MAE(μs'):")
        print(f"  run_name: {best_mae_mus_run['run_name']}")
        print(f"  channel_mode: {best_mae_mus_run.get('channel_mode')}")
        print(f"  MAE(μs'): {best_mae_mus_val:.6f}")
    else:
        print("\nBest by MAE(μs'): (no runs with MAE)")

    # Best RMSE μa
    if best_rmse_mua_run is not None:
        print("\nBest by RMSE(μa):")
        print(f"  run_name: {best_rmse_mua_run['run_name']}")
        print(f"  channel_mode: {best_rmse_mua_run.get('channel_mode')}")
        print(f"  RMSE(μa): {best_rmse_mua_val:.6f}")
    else:
        print("\nBest by RMSE(μa): (no runs with RMSE)")

    # Best RMSE μs'
    if best_rmse_mus_run is not None:
        print("\nBest by RMSE(μs'):")
        print(f"  run_name: {best_rmse_mus_run['run_name']}")
        print(f"  channel_mode: {best_rmse_mus_run.get('channel_mode')}")
        print(f"  RMSE(μs'): {best_rmse_mus_val:.6f}")
    else:
        print("\nBest by RMSE(μs'): (no runs with RMSE)")


def main():
    runs = load_run_logs(JSON_PATH)
    print(f"Loaded {len(runs)} runs from log.")
    print_global_best_metrics(runs)


if __name__ == "__main__":
    main()


Loaded 192 runs from log.
GLOBAL BEST CONFIGS BY METRIC

Best by MSE (best_val):
  run_name: early_mid_late_w11_o4_lr0.001
  channel_mode: early_mid_late
  sg_window: 11, sg_order: 4
  lr: 0.001
  best_val (MSE): 0.020709

Best by MAE(μa):
  run_name: early_mid_late_w31_o2_lr0.0003
  channel_mode: early_mid_late
  MAE(μa): 0.011018

Best by MAE(μs'):
  run_name: hybrid_4ch_w11_o1_lr0.001
  channel_mode: hybrid_4ch
  MAE(μs'): 0.150076

Best by RMSE(μa):
  run_name: early_mid_late_w31_o2_lr0.0003
  channel_mode: early_mid_late
  RMSE(μa): 0.013267

Best by RMSE(μs'):
  run_name: hybrid_4ch_w31_o3_lr0.0003
  channel_mode: hybrid_4ch
  RMSE(μs'): 0.228337
